# 🤖 Formation RAG – Notebook intégral

---
## 🎯 Objectifs pédagogiques
- Comprendre chaque composant d’un **RAG**
- Manipuler le code dans Colab et exporter `app_base.py`
- Installer **Ollama** et les modèles requis
- Tester un prototype sur un PDF d’exemple
- Découvrir les étapes de mise en production (Partie 2)

> Les cellules `EXPORT` écrivent dans *app_base.py* via `%%writefile -a`. Vous pourrez ainsi le versionner, mais **on n’utilisera plus ce fichier en Partie 2** (remplacé par `app_finale.py`).


## 🚧 Séquence 1.0 – Installation d’Ollama & des modèles

In [ ]:
# @title 🛠️ Installation Ollama (linux/Colab)
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# @title 📥 Téléchargement des modèles
!ollama pull llama3.2
!ollama pull nomic-embed-text

## 🔗 Séquence 1.1 – Bootstrap Colab

In [ ]:
# @title 🔌 Connexion Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# @title 📥 Clone du dépôt
%cd /content/gdrive/MyDrive
!git clone https://github.com/antoinecstl/FormationEYAI.git
%cd FormationEYAI

In [ ]:
# @title 🛠️ Installation des dépendances
!pip install -q -r requirements.txt

## 🔍 Séquence 1.2 – Bases du RAG : embeddings & similarité

In [ ]:
%%writefile -a app_base.py
import numpy as np, ollama

EMBED_MODEL = "nomic-embed-text:latest"

def embed_texts(texts):
    """Retourne un np.ndarray shape (n, d)"""
    return np.array([ollama.embeddings(model=EMBED_MODEL, prompt=t)['embedding'] for t in texts], dtype='float32')


In [ ]:
# @title 🔬 Test d'embedding
phrase1 = "Le soleil brille"
phrase2 = "Aujourd'hui il fait beau"
# @markdown `phrase1` et `phrase2` sont des phrases à comparer.
# @markdown La similarité entre les phrases est calculée en utilisant le produit scalaire des
# @markdown embeddings normalisés de chaque phrase.
# @markdown La similarité est un nombre entre -1 et 1, où 1 signifie que les phrases sont très similaires,
# @markdown 0 signifie qu'elles ne sont pas similaires, et -1 signifie qu'elles sont opposées.
# @markdown Vous pouvez modifier les phrases pour tester d'autres exemples.
# @markdown Exécutez la cellule pour voir le résultat.
vecs = embed_texts([phrase1, phrase2])
sim = float(vecs[0] @ vecs[1] / (np.linalg.norm(vecs[0])*np.linalg.norm(vecs[1])))
print(f"Similarité : {sim:.3f}")

## 📐 Séquence 1.3 – Chunking & nettoyage d’un PDF d’exemple

In [ ]:
%%writefile -a app_base.py
from langchain.text_splitter import RecursiveCharacterTextSplitter

def auto_chunk_size(tok:int)->int:
    return 1024 if tok<8000 else 768 if tok<20000 else 512

def chunk_document(text:str):
    size=auto_chunk_size(len(text.split()))
    splitter=RecursiveCharacterTextSplitter(
        separators=["\n\n","\n",". "],
        chunk_size=size,
        chunk_overlap=size//4,
        length_function=len,
        )
    return [c for c in splitter.split_text(text) if len(c)>100]


In [ ]:
# @title 📖 Chargement du PDF d'exemple
from PyPDF2 import PdfReader, PdfReadError

sample_path = "/rapport.pdf"  # fourni dans le repo
pages = PdfReader(sample_path).pages
full_text = "\n".join(p.extract_text() or "" for p in pages)

print(f"📄 Le document contient {len(full_text.split())} tokens environ.")

In [ ]:
# @title 🌳 Chunking du PDF
chunks = chunk_document(full_text)
print(f"🌳 {len(chunks)} chunks créés. Aperçu :\n{chunks[0][:300]}…")

## 📊 Séquence 1.4 – Index vectoriel FAISS

In [ ]:
%%writefile -a app_base.py
import faiss, numpy as np

def build_faiss_index(vectors:np.ndarray)->faiss.IndexFlatIP:
    d=vectors.shape[1]
    idx=faiss.IndexFlatIP(d)
    idx.add(vectors.astype('float32'))
    return idx


In [ ]:
# @title 🧪 Construction index chunks
import numpy as np

chunk_vecs = embed_texts(chunks)
index = build_faiss_index(chunk_vecs)

print(index.ntotal, "vecteurs dans l'index")

## 🧮 Séquence 1.5 – Algorithme MMR

In [ ]:
%%writefile -a app_base.py
from sklearn.metrics.pairwise import cosine_similarity

def mmr(query_vec:np.ndarray, cand:np.ndarray, k:int=5, λ:float=0.3):
    selected, rest = [], list(range(len(cand)))
    while len(selected)<min(k,len(rest)):
        best, best_score = None, -1e9
        for idx in rest:
            sim_q = float(query_vec @ cand[idx]/(np.linalg.norm(query_vec)*np.linalg.norm(cand[idx])+1e-6))
            sim_s = max(cosine_similarity(cand[idx][None,:], cand[selected])[0]) if selected else 0.
            score = λ*sim_q - (1-λ)*sim_s
            if score>best_score:
                best, best_score = idx, score
        selected.append(best); rest.remove(best)
    return selected


In [ ]:
# @title 🔬 Test MMR
import numpy as np

q_vec = embed_texts(["Sujet principal du rapport ?"])[0]
sel = mmr(q_vec, chunk_vecs, 3)
print(sel)

## 🧑‍🎤 Séquence 1.6 – Prompt engineering

In [ ]:
%%writefile -a app_base.py
def build_prompt(question:str, ctxs:list[str]):
    ctx_block="\n\n".join(f"[{i+1}] {c}" for i,c in enumerate(ctxs))
    system="Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. Citez les sources [n]."
    return [
        {"role":"system","content":system},
        {"role":"user","content":f"CONTEXTE(S):\n{ctx_block}\n\nQUESTION: {question}\n\nRéponse:"}
    ]


In [ ]:
# @title 🔬 Prompt test (sans LLM pour l'instant)
print(build_prompt("Pourquoi le ciel est bleu ?", ["La diffusion Rayleigh explique la couleur du ciel."]))

## 🔗 Séquence 1.7 – Assemblage mini‑RAG (prototype)

In [ ]:
%%writefile -a app_base.py
def ask(question:str, chunks:list[str], vecs:np.ndarray, top_k:int=3):
    q_vec = embed_texts([question])[0]
    idx = build_faiss_index(vecs)
    _, I = idx.search(q_vec[None,:], top_k)
    ctx = [chunks[i] for i in I[0]]
    prompt = build_prompt(question, ctx)
    # ⬇️ Remplacez ce stub par un vrai appel LLM si besoin
    return "[Stub] Réponse générée.\nContextes utilisés :" + str(I[0])


In [ ]:
# @title 🧪 Prototype RAG sur le PDF
question = "Quel est le thème principal de ce document ?"
print(ask(question, chunks, chunk_vecs))

### 📦 EXPORT – Générer `app_base.py`

In [ ]:
# @title 🚚 Vérif contenu app_base.py
!echo '--- Lignes et poids de app_base.py --'
!wc -l app_base.py
!du -h app_base.py

---
# 🛠️ Partie 2 – Mise en production avec `app_finale.py`

> À partir d’ici **on n’utilise plus `app_base.py`**.  
> Tout le travail se fait avec `app_finale.py` fourni dans le dépôt.


## 🏗️ Séquence 2.1 – Préparer l’environnement

In [ ]:
# @title ⚙️ Optionnel : créer un virtualenv local
# !python -m venv venv && source venv/bin/activate

## 📦 Séquence 2.2 – Récupérer `app_finale.py`

In [ ]:
# @title 📥 Copier le script final
%cp FormationEYAI/app_finale.py ./app_finale.py
!ls -l app_finale.py

## 🤖 Séquence 2.3 – Démarrer Ollama en production

In [ ]:
# @title 🚀 Lancement Ollama en arrière‑plan
import subprocess, time, os, signal
ollama_proc = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)
print('✅ Ollama est prêt')

## 🖥️ Séquence 2.4 – Lancer l’application Streamlit

In [ ]:
# @title 🎛️ Run Streamlit + LocalTunnel
!pip install -q streamlit localtunnel
!streamlit run app_finale.py &>/content/logs.txt & npx localtunnel --port 8501

## 📈 Séquence 2.5 – Observabilité

- Temps de réponse, logs Streamlit (`tail -f content/logs.txt`)  
- Sécurité des données
- Coût GPU / CPU


## 🎓 Mini‑projet final

Ajoutez un **second modèle d’embedding** ou la prise en charge d’un format `.docx`, puis présentez vos résultats.
